In [1]:
import os
import django

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "portfolio_management.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django.setup()

from common.models import Brokers, Prices, Transactions, FX, Assets
from users.models import CustomUser
from utils import NAV_at_date, Irr, chart_dates, chart_labels, chart_colour, portfolio_at_date, calculate_security_nav, currency_format, calculate_portfolio_value
from datetime import date
from django.db.models import F, Sum

In [2]:
selected_brokers = broker_ids = [2]
effective_current_date = date.today()
currency_target = 'USD'
number_of_digits = 2

In [3]:
from pyxirr import xirr


def Irr(date, currency, asset_id=None, broker_id_list=None, start_date=''):
    
    # Calculate portfolio value
    portfolio_value = calculate_portfolio_value(date, currency, asset_id, broker_id_list)

    if portfolio_value < 0:
        return 'N/R'
    
    cash_flows = []
    transaction_dates = []

    # Collect cash flows and transaction dates for the portfolio
    transactions = Transactions.objects.filter(date__lte=date, security_id=asset_id)
    
    # if asset_id:
    #     transactions = transactions.filter(security_id=asset_id)

    if broker_id_list:
        transactions = transactions.filter(broker_id__in=broker_id_list)

    if start_date:
        transactions = transactions.filter(date__gte=start_date)

    # print(f"utils.py. line 123. Transactions: {transactions}")

    # Calculate cash flows and transaction dates
    for transaction in transactions:
        print(f"utils.py. line 127. Transaction details: {transaction.quantity}")
        fx_rate = FX.get_rate(transaction.currency.upper(), currency, transaction.date)['FX']

        if transaction.type == 'Cash in' or transaction.type == 'Cash out':
            cash_flow = -transaction.cash_flow
        else:
            cash_flow = transaction.cash_flow or (-transaction.quantity * transaction.price + (transaction.commission or 0))
        
        cash_flows.append(round(cash_flow * fx_rate, 2))
        transaction_dates.append(transaction.date)

    # Calculate start portfolio value if provided
    if start_date:
        start_portfolio_value = calculate_portfolio_value(start_date, currency, asset_id, broker_id_list)
        cash_flows.insert(0, -start_portfolio_value)
        transaction_dates.insert(0, start_date)

    # Calculate portfolio value
    portfolio_value = calculate_portfolio_value(date, currency, asset_id, broker_id_list)

    # Append end portfolio value to cash flows and dates
    cash_flows.append(portfolio_value)
    transaction_dates.append(date)

    print(f"utils. line 146. Cash flows: {cash_flows}, Transaction dates: {transaction_dates}")

    try:
        return round(xirr(transaction_dates, cash_flows), 4)
    except:
        return 'N/A'

# def calculate_portfolio_value(date, currency=None, asset_id=None, broker_id_list=None):
#     portfolio_value = 0

#     if asset_id is None:
#         portfolio_value = NAV_at_date(broker_id_list, date, currency, [])['Total NAV']
#     else:
#         asset = Assets.objects.get(id=asset_id)
#         # print(f"utils.py. line 165. asset current price: {asset.current_price(date)}")
#         # print(f"utils.py. line 167. Asset data: {asset.current_price(date).price}, {asset.position(date, broker_id_list)}")
#         portfolio_value = round(asset.price_at_date(date, currency).price * asset.position(date, broker_id_list), 2)

#     return portfolio_value

security_id = 4
security = Assets.objects.get(id=security_id)
print('Price:', security.price_at_date(effective_current_date).price)
print('Value:', calculate_portfolio_value(effective_current_date, security.currency, security_id))

Irr(effective_current_date, security.currency, security_id)

Price: 165.00
Value: 16500.00
utils.py. line 127. Transaction details: 50.00
utils.py. line 127. Transaction details: None
utils.py. line 127. Transaction details: 50.00
utils. line 146. Cash flows: [Decimal('-5000.36'), Decimal('0.76'), Decimal('-7500.50'), Decimal('16500.00')], Transaction dates: [datetime.date(2020, 6, 2), datetime.date(2020, 6, 25), datetime.date(2020, 9, 1), datetime.date(2024, 5, 13)]


0.0758

In [26]:
totals = ['entry_value', 'current_value', 'realized_gl', 'unrealized_gl', 'capital_distribution', 'commission', 'total_return']
categories = ['current_value', 'realized_gl', 'commission', 'investment_date']


p = list(set(totals) & set(categories)) + ['entry_value', 'total_return_amount']

# totals and categories + ['entry_value', 'total_return_amount']
print(p)

['commission', 'realized_gl', 'current_value', 'entry_value', 'total_return_amount']
